In [ ]:
import import_ipynb
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score


from utilities.UtilityFunctions import retrieve_reviews_df, preprocessing, vectorize, text_preprocess, retrieve_and_preprocess

In [ ]:
df = retrieve_and_preprocess()

In [3]:
tags = ['html', 'stop','punc','lower']
text_preprocess(df, tags)

In [5]:
X = df['Text']
y = df['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(
     X, y, 
     test_size=0.2, 
     random_state=42
     )
X_counts, X_tfidf = vectorize(X_train)


In [ ]:
#scores = []
#for k in range(1, 151):
#    rfc_pipe = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf_randomForest", RandomForestClassifier(n_estimators=k, verbose=2))])
#    rfc_pipe.fit(X_train, y_train)
#    y_pred = rfc_pipe.predict(X_test)
#    scores.append(accuracy_score(y_test, y_pred))

In [ ]:
#plt.plot(range(1, 151), scores)
#plt.xlabel('Value of n_estimators for Random Forest Classifier')
#plt.ylabel('Testing Accuracy')

In [6]:
clf_randomForest_pipe = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf_randomForest", RandomForestClassifier(verbose=2))])
clf_randomForest_pipe.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   39.9s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100


In [ ]:
predictedRandomForest = clf_randomForest_pipe.predict(X_test)
np.mean(predictedRandomForest == y_test)